<a href="https://colab.research.google.com/github/akashbalor123/Machine-Learning/blob/main/Fake_News_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import all the Packages which is required for this process

In [136]:
import pandas as pd
import matplotlib.pyplot as plt
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [137]:
import nltk


In [138]:
data_real = pd.read_csv('/content/true.csv')
data_fake = pd.read_csv('/content/fake.csv')

In [139]:
data_real.info()
data_fake.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21417 entries, 0 to 21416
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    21417 non-null  object
 1   text     21417 non-null  object
 2   subject  21417 non-null  object
 3   date     21417 non-null  object
dtypes: object(4)
memory usage: 669.4+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23481 entries, 0 to 23480
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    23481 non-null  object
 1   text     23481 non-null  object
 2   subject  23481 non-null  object
 3   date     23481 non-null  object
dtypes: object(4)
memory usage: 733.9+ KB


In [140]:
data_fake["label"] = 0
data_real["label"] = 1

In [141]:
data = pd.concat([data_real, data_fake],axis=0)

In [142]:
data

,title,text,subject,date,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1
...,...,...,...,...,...
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016",0
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016",0
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016",0
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016",0


In [143]:
data = data.sample(frac=1).reset_index(drop = True)
data

,title,text,subject,date,label
0,Senate Republicans struggle to salvage healthc...,WASHINGTON (Reuters) - The top U.S. Senate Rep...,politicsNews,"June 28, 2017",1
1,SEAN SPICER CALLS OUT Race Baiting Journalist:...,,politics,"Mar 28, 2017",0
2,Virginia School Has Black Kid Arrested For Dr...,Seriously. This actually happened in America.M...,News,"September 30, 2016",0
3,Incoming Trump Deputy Sec. Of State Blames Ru...,If you ve ever wondered if Donald Trump is com...,News,"December 12, 2016",0
4,Kellyanne Lied When She Said She Said She ‘Mi...,Among the never ending stream of Trump adminis...,News,"February 6, 2017",0
...,...,...,...,...,...
44893,Irish parliament committee backs referendum on...,DUBLIN (Reuters) - An Irish parliamentary comm...,worldnews,"December 13, 2017",1
44894,The Democratic Party Has Never Done This Befo...,The draft version of the Democratic Party plat...,News,"July 1, 2016",0
44895,Senate takes up broadband privacy repeal,WASHINGTON (Reuters) - The U.S. Senate on Wedn...,politicsNews,"March 22, 2017",1
44896,Senators criticize fraud in U.S. low-income ph...,WASHINGTON (Reuters) - A top U.S. Senate Repub...,politicsNews,"September 14, 2017",1


In [144]:
data['text'] = data.title + ' ' + data.text

In [145]:
data = data[['text', 'label']]

In [146]:
data

,text,label
0,Senate Republicans struggle to salvage healthc...,1
1,SEAN SPICER CALLS OUT Race Baiting Journalist:...,0
2,Virginia School Has Black Kid Arrested For Dr...,0
3,Incoming Trump Deputy Sec. Of State Blames Ru...,0
4,Kellyanne Lied When She Said She Said She ‘Mi...,0
...,...,...
44893,Irish parliament committee backs referendum on...,1
44894,The Democratic Party Has Never Done This Befo...,0
44895,Senate takes up broadband privacy repeal WASHI...,1
44896,Senators criticize fraud in U.S. low-income ph...,1


In [147]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [148]:
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

In [149]:
data['text']

,text
0,Senate Republicans struggle to salvage healthc...
1,SEAN SPICER CALLS OUT Race Baiting Journalist:...
2,Virginia School Has Black Kid Arrested For Dr...
3,Incoming Trump Deputy Sec. Of State Blames Ru...
4,Kellyanne Lied When She Said She Said She ‘Mi...
...,...
44893,Irish parliament committee backs referendum on...
44894,The Democratic Party Has Never Done This Befo...
44895,Senate takes up broadband privacy repeal WASHI...
44896,Senators criticize fraud in U.S. low-income ph...


In [150]:
port_stem = PorterStemmer()

In [151]:
stop_words = set(stopwords.words('english'))  # Do this ONCE, outside the function!

def stemming(text):
    stemmed_content = re.sub('[^A-Za-z ]','', text)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if word not in stop_words]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content


In [152]:
# for a progress bar (optional, for visual feedback)
from tqdm import tqdm
tqdm.pandas()

data['text'] = data['text'].progress_apply(stemming)

100%|██████████| 44898/44898 [03:56<00:00, 189.92it/s]
/tmp/ipython-input-1016173291.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['text'] = data['text'].progress_apply(stemming)


In [153]:
data

,text,label
0,senat republican struggl salvag healthcar effo...,1
1,sean spicer call race bait journalist stop sha...,0
2,virginia school black kid arrest drink milk se...,0
3,incom trump deputi sec state blame russian hac...,0
4,kellyann lie said said misspok bowl green mass...,0
...,...,...
44893,irish parliament committe back referendum abor...,1
44894,democrat parti never done big victori human ri...,0
44895,senat take broadband privaci repeal washington...,1
44896,senat critic fraud us lowincom phone program w...,1


In [154]:
vectorize = TfidfVectorizer()
X = vectorize.fit_transform(data['text'])
y = data['label']

In [155]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [156]:
log = LogisticRegression()
log.fit(X_train, y_train)

LogisticRegression()

In [157]:
y_pred = log.predict(X_test)
accuracy = accuracy_score(y_pred, y_test)
accuracy

0.9859620705945873

In [158]:
report = classification_report(y_test , y_pred , output_dict=True)
Report = pd.DataFrame(report)
Report

,0,1,accuracy,macro avg,weighted avg
precision,0.986942,0.984891,0.985962,0.985917,0.985963
recall,0.986177,0.985726,0.985962,0.985952,0.985962
f1-score,0.986560,0.985309,0.985962,0.985934,0.985962
support,7741.000000,7076.000000,0.985962,14817.000000,14817.000000


In [159]:
new_data = ['India is a developing country']
new_data_vector = vectorize.transform(new_data)
new_pred = log.predict(new_data_vector)
new_pred

array([0])